In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import os
from PIL import Image
from matplotlib import patches
import time
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import cv2
from pathlib import Path
from collections import Counter
import albumentations as al
from albumentations.pytorch import ToTensor, ToTensorV2

#bokeh.io.output_notebook(INLINE) 

In [ ]:
total = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
total.head()

In [ ]:
#lets see some images
train_dir = "../input/cassava-leaf-disease-classification/train_images/"
test_dir = "../input/cassava-leaf-disease-classification/test_images/"

train_images = glob(train_dir + "*")
test_images = glob(test_dir + "*")

print("Total train images are ",len(train_images))

print("Total test images are ",len(test_images))

disease = {"0":"Cassava Bacterial Blight (CBB)",
"1":"Cassava Brown Streak Disease (CBSD)",
"2":"Cassava Green Mottle (CGM)",
"3":"Cassava Mosaic Disease (CMD)",
"4":"Healthy"
}

Train images are in ok, but we have nothing to check it on

In [ ]:
#Lets convert 18000 to train images and then rest to valid 
train = total[:18000]
valid = total[18000:]

In [ ]:
train.head()

In [ ]:
train["disease"] = train["label"].astype(str).map(disease)


In [ ]:
train.head()

In [ ]:
le = LabelEncoder()
le.fit(train['label'])
train['label'] = le.transform(train['label'])

In [ ]:
plt.figure(figsize=(8,4))
sns.countplot(y="disease",data=train)


In [ ]:
tes = train[:9]

In [ ]:
plt.figure(figsize=(30,25))
for iterator,filename in tes.iterrows():
    images = Image.open(os.path.join(train_dir,filename["image_id"]))
    plt.subplot(3,3,iterator+1)
    plt.imshow(images,cmap=plt.cm.bone)
    plt.title( str(filename["disease"]),fontsize=15)
    
plt.tight_layout()

Let's look at the dimension of 300 images

In [ ]:
shape = Counter()
for idx, fig in train[:300].iterrows():
    image =cv2.imread(os.path.join(train_dir,fig['image_id']))
    
    shape.update(image.shape)
 
print(shape)
print(image.shape)
    
    
    

As we can see, starting from top most of the images till 300 are the same shape, 600 x 800 X 3

In [ ]:
CBB = train[train['disease']=="Cassava Bacterial Blight (CBB)"].reset_index(drop=True)


In [ ]:
def dis_plot(df):
    plt.figure(figsize=(30,25))
    df.reset_index(drop=True)
    for idx,row in df.iterrows():
        image = Image.open(os.path.join(train_dir,row["image_id"]))
        plt.subplot(3,2,idx+1)
        plt.imshow(image,cmap=plt.cm.bone)
        

#                     CASSAVA BACTERIAL BLIGHT

# **There are total 924 images in this class**

In [ ]:
dis_plot(CBB[:4])

# **Cassava Brown Streak Disease **

# **There are total 1833 images in this class**

In [ ]:
CBSD = train[train["disease"]=="Cassava Brown Streak Disease (CBSD)"].reset_index(drop=True)
dis_plot(CBSD[:6])

#  Cassava Green Mottle 

# **There are total 2036 images in this class**

In [ ]:
CGM = train[train["disease"]=="Cassava Green Mottle (CGM)"].reset_index(drop=True)
dis_plot(CGM[:6])

# **Cassava Mosaic Disease**

# **There are 11054 images in this class**

In [ ]:
CMD = train[train["disease"]=="Cassava Mosaic Disease (CMD)"].reset_index(drop=True)
dis_plot(CMD[:6])


# **Healthy**

# **There are total 2153 healthy leaves**

In [ ]:
healthy = train[train["disease"]=="Healthy"].reset_index(drop=True)
dis_plot(healthy[:6])

In [ ]:
train['label'].value_counts()

In [ ]:
mean = [0.4589, 0.5314, 0.3236]
std = [0.2272, 0.2297, 0.2200]

In [ ]:
import torch
from torch.utils.data import DataLoader,Dataset


In [ ]:
#using transforms
train_transform = al.Compose([
    al.RandomResizedCrop(256,256),
    al.HorizontalFlip(p=0.5),
    al.ShiftScaleRotate(p=0.5),
    al.HueSaturationValue(
    hue_shift_limit=0.2,
    sat_shift_limit=0.2,
    val_shift_limit=0.2,
    p=0.5),
    
    al.RandomBrightnessContrast(
    brightness_limit=(-0.1,0.1),
    contrast_limit=(-0.1,0.1),
    p=0.5),
    
    al.Normalize(
    mean=mean,
    std=std,
    max_pixel_value=255,
    p=1.0),

    al.CoarseDropout(p=0.5),
    al.Cutout(p=0.5),
ToTensorV2()],p=1.0)

valid_transform = al.Compose([
    al.CenterCrop(256,256,p=1.0),
    al.Resize(256,256),
    al.Normalize(
    mean=mean,
    std=std,
    max_pixel_value=255,
    p=1.0),
    ToTensorV2()
    
],p=1.0)

In [ ]:
train[:20]["image_id"].values
path =Path("../input/cassava-leaf-disease-classification")
train_path =path/"train_images" 

In [ ]:
class LeafData(Dataset):
    def __init__(self, df, split="train"):
        if split == "train":
            self.transforms = train_transform
        elif split == "val":
            self.transforms = valid_transform
            
        self.paths = [train_path/id_ for id_ in df['image_id'].values]
        self.labels = df['label'].values
    
    def __getitem__(self, idx):
        img = cv2.imread(str(self.paths[idx]))[..., ::-1] #
        img = self.transforms(image=img)['image']
        label = self.labels[idx]
       
        return img, label
    
    def __len__(self):
        return len(self.paths)

def make_dataloaders(batch_size=32, num_workers=4, pin_memory=True, **kwargs):
    dataset = LeafData(**kwargs)
    dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers,
                            pin_memory=pin_memory, shuffle=True if kwargs['split'] == "train" else False)
    return dataloader

In [ ]:
train_dl = make_dataloaders(df=train,split="train")
val_dl = make_dataloaders(df=valid,split="valid")
x,y = next(iter(train_dl))
x.shape,y.shape,x.mean(dim=(0,2,3)),x.std(dim=(0,2,3))

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def accuracy(preds,target):
    preds = preds.argmax(dim=1)
    return (preds==target).float().mean()

def one_epoch(model,dl,loss_func,opt=None,lr_schedule=None):
    running_loss=0
    running_acc =0
    
    for xb,yb in tqdm(dl):
        xb,yb = xb.to(device),yb.to(device)
        pred = model(xb)
        loss = loss_func(pred,yb)
        
        if opt is not None:
            opt.zero_grad()
            loss.backward()
            opt.step()
            if lr_scheduler is not None:
                lr_scheduler.step()
            
        running_acc+=accuracy(preds,yb)
        running_loss+=loss.iten()
    
    return running_loss/len(dl), running_acc/len(dl) 